In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfTransformer,TfidfVectorizer
import nltk

In [2]:
test = pd.read_csv('test.csv',encoding='utf-8')
train= pd.read_csv('train.csv', encoding='utf-8')

In [3]:
train_x = train.iloc[:,0]
train_y = train.iloc[:,1]

In [4]:
from sklearn.preprocessing import LabelEncoder
lbl = LabelEncoder()
train_y = lbl.fit_transform(train_y)

In [5]:
#先'/'再'-'再'.'再'com'
def getTokens(input):
    tokensBySlash = str(input.encode('utf-8')).split('/')	#get tokens after splitting by slash
    allTokens = []
    for i in tokensBySlash:
        tokens = str(i).split('-')	#get tokens after splitting by dash
        tokensByDot = []
        for j in range(0,len(tokens)):
            tempTokens = str(tokens[j]).split('.')	#get tokens after splitting by dot
            tokensByDot = tokensByDot + tempTokens
        allTokens = allTokens + tokens + tokensByDot
    allTokens = list(allTokens)#-remove_set--#remove redundant tokens
    if 'com' in allTokens:
        allTokens.remove('com')	#removing .com since it occurs a lot of times and it should not be included in our features
    return allTokens

def preprocessing_data_2(list_data):
    new_data = ''
    for i in list_data:
        words = nltk.tokenize.TweetTokenizer(strip_handles=False, reduce_len = True).tokenize(i)
        temp = ' '.join(words)
        new_data = new_data + ' '+temp
    return new_data

def pipeline_x(data):
    data = getTokens(data)
    data = preprocessing_data_2(data)
    return data


In [6]:
pre_data = train_x.apply(lambda x:pipeline_x(x))

In [10]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfTransformer,TfidfVectorizer

print('TfidfVectorizer')
tf = TfidfVectorizer(ngram_range=(1,2),analyzer='char')
discuss_tf = tf.fit_transform(pre_data)
 
print('HashingVectorizer')
ha = HashingVectorizer(ngram_range=(1,1),lowercase=False)
discuss_ha = ha.fit_transform(pre_data)
 

TfidfVectorizer
HashingVectorizer


In [11]:
from scipy.sparse import csr_matrix, hstack
data = hstack((discuss_tf,discuss_ha)).tocsr()

In [12]:
#对数据进行分割
#先打乱
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = train_y[indices]

train_sample = data.shape[0]  - (data.shape[0] // 10)
print('sum',data.shape[0])
print('indexs',train_sample)

x_train = data[:train_sample]
y_train = labels[:train_sample]
x_val = data[train_sample:]
y_val = labels[train_sample:]

sum 420464
indexs 378418


In [13]:
print(x_train.shape)
print(x_val.shape)

(378418, 1050478)
(42046, 1050478)


In [14]:
import pandas as pd
import numpy as np
import jieba
import jieba.analyse
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfTransformer,TfidfVectorizer
 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold,StratifiedKFold
from scipy.sparse import csr_matrix, hstack
from sklearn.metrics import f1_score,accuracy_score,classification_report
import sklearn.metrics as metrics
from sklearn import svm
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import pickle as pk
from sklearn import svm

In [16]:
N = 10
kf = StratifiedKFold(n_splits=N, random_state=2018).split(data,labels)

In [18]:
y_train_oofp = np.zeros_like(y_train, dtype='float32')
y_train_oofp1 = np.zeros_like(y_train, dtype='float32')
 
y_test_oofp = np.zeros((test.shape[0], N))
y_test_oofp_1 = np.zeros((test.shape[0], N))

In [19]:
from sklearn.svm import LinearSVC

clf = LinearSVC(C=0.6, loss='hinge', tol=1e-4)

In [22]:
def micro_avg_f1(y_true, y_pred):
    return metrics.f1_score(y_true, y_pred, average='micro')

In [23]:
acc = 0
vcc = 0
for i ,(train_fold,test_fold) in enumerate(kf):
    X_train, X_validate, label_train, label_validate = data[train_fold, :], data[test_fold, :], labels[train_fold], labels[test_fold]
    clf.fit(X_train, label_train)
    
    val_ = clf.predict(X_validate)
    y_train_oofp[test_fold] = val_
    print('sentiment_value_f1:%f' % micro_avg_f1(label_validate, val_))
    acc += micro_avg_f1(label_validate, val_)

 
print(acc/N)

F:\Anaconda\envs\tensorflow-gpu\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


sentiment_value_f1:0.981497
sentiment_value_f1:0.981639
sentiment_value_f1:0.981853
sentiment_value_f1:0.981140
sentiment_value_f1:0.982210
sentiment_value_f1:0.981925


IndexError: index 378428 is out of bounds for axis 0 with size 378418